In [43]:
import numpy as np
import matplotlib
import pandas as pd
import tensorflow as tf

dataset = pd.read_csv('/Users/wzhang/Downloads/lake_erie_habs_W4_6_8_9_12_13_16_2013-2020.xlsx - HAB_data (1).csv')

train = dataset.iloc[:743,:]
test = dataset.iloc[743:,:]
print(train)
print(test)
X_train = pd.DataFrame(train.iloc[:,4:15].values)
Y_train = train.iloc[:,16:].values
print(X_train.shape)
print(Y_train.shape)
X_test = pd.DataFrame(test.iloc[:,4:15].values)
Y_test = test.iloc[:,16:].values
print(X_test.shape)
print(Y_test.shape)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



          Date  Site  Latitude (decimal deg)  Longitude (decimal deg)  \
0    5/29/2013   WE6                 41.7134                 -83.3804   
1    5/29/2013   WE8                 41.8343                 -83.3633   
2    5/29/2013   WE4                 41.8267                 -83.1930   
3    6/10/2013   WE6                 41.7144                 -83.3805   
4    6/10/2013   WE8                 41.8353                 -83.3631   
..         ...   ...                     ...                      ...   
738  10/7/2019  WE16                 41.6599                 -83.1467   
739  10/7/2019  WE13                 41.7428                 -83.1354   
740  10/7/2019  WE12                 41.7023                 -83.2639   
741  10/7/2019   WE4                 41.8270                 -83.1949   
742  6/16/2020   WE6                 41.7054                 -83.3864   

     Secchi Depth (m)  CTD Temperature (°C)  \
0                1.60                  18.5   
1                1.00        

In [44]:
from keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dropout, Dense, BatchNormalization, GRU, LayerNormalization, MultiHeadAttention,TimeDistributed, Input, Flatten
from sklearn.ensemble import RandomForestRegressor

In [45]:
def create_rf():
    # Initialize Random Forest regressor
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

    # Train the model
    rf_model.fit(X_train, Y_train)
    return rf_model




In [46]:
from sklearn.svm import SVR
def create_SVM():
    # Initialize SVM regressor
    svm_model = SVR(kernel='rbf', C=10000, gamma='scale')

    # Train the model
    svm_model.fit(X_train, Y_train)
    return svm_model


In [47]:
rf = create_rf()
svm = create_SVM()

# ridge = create_ridge()
# svm = create_SVM()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [48]:
from tensorflow.keras.layers import concatenate


# gruPredict = gru.predict(X_train)
# lassoPredict = lasso.predict(X_train)
# lstmPredict = lstm.predict(X_train)
rfPredict = rf.predict(X_train)
svmPredict = svm.predict(X_train)
# ridgePredict = ridge.predict(X_train)
# svmPredict = svm.predict(X_train)
# rfPredict = rf.predict(X_train)
# lassoPredict.resize(671,1)
rfPredict.resize(743,1)
svmPredict.resize(743,1)
# svmPredict.resize(671,1)
trainData = concatenate([rfPredict,svmPredict])

In [49]:
from sklearn.ensemble import AdaBoostRegressor

In [50]:
regressor = AdaBoostRegressor(n_estimators=1000, random_state=0)
regressor.fit(trainData, Y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostRegressor(n_estimators=1000, random_state=0)

In [51]:
# gruPredictTest = gru.predict(X_test)
# lassoPredictTest = lasso.predict(X_test)
# lstmPredictTest = lstm.predict(X_test)
rfPredictTest = rf.predict(X_test)
svmPredictTest = svm.predict(X_test)
# ridgePredictTest = ridge.predict(X_test)
# svmPredictTest = svm.predict(X_test)
# svmPredictTest.resize(168,1)
# svmPredictTest.resize(96,1)
# lassoPredictTest.resize(168,1)
rfPredictTest.resize(96,1)
svmPredictTest.resize(96,1)
testData = concatenate([rfPredictTest, svmPredictTest])

In [52]:
res = regressor.predict(testData)
print(res)

[10.40925054 14.75445476 10.40925054 10.40925054 10.40925054 32.2938961
 10.40925054 10.40925054 10.40925054 32.2938961  10.40925054 32.2938961
 10.40925054 32.2938961  10.40925054 44.84174107 10.40925054 44.84174107
 10.40925054 33.39144279 10.40925054 56.84664179 10.40925054 10.40925054
 10.40925054 10.40925054 56.84664179 14.58531507 56.84664179 10.40925054
 10.40925054 32.2938961  10.40925054 33.39144279 14.58531507 44.84174107
 10.40925054 10.40925054 32.2938961  10.40925054 44.84174107 14.75445476
 66.61938272 18.69404011 10.40925054 14.75445476 10.40925054 44.84174107
 44.84174107 66.61938272 14.58531507 10.40925054 10.40925054 10.40925054
 23.8544507  32.2938961  44.84174107 10.40925054 10.40925054 14.75445476
 10.40925054 44.84174107 10.40925054 33.39144279 10.40925054 10.40925054
 10.40925054 10.40925054 32.2938961  18.69404011 14.75445476 10.40925054
 10.40925054 14.75445476 10.40925054 32.2938961  23.8544507  32.2938961
 10.40925054 10.40925054 14.75445476 10.40925054 10.40

In [53]:
from sklearn.metrics import r2_score 
r2 = r2_score(Y_test, res) 
print(r2)

0.7612029775020166


In [54]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(Y_test,res)
print(mae)

6.537390474886408


In [55]:
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(Y_test,res)
print(mape)

0.7369097758303856


In [56]:
from sklearn.metrics import root_mean_squared_error
rmse = root_mean_squared_error(Y_test,res)
print(rmse)

8.944763308905456
